In [1]:
import requests
import pandas as pd
import numpy as np
import pickle
from numpy.random import random_sample

In [2]:
def save_pkl(file_dir, data):
    f = open(file_dir,"wb")
    pickle.dump(data, f, protocol=4)
    f.close()
    
def read_pkl(file_dir):
    f = open(file_dir,"rb")
    data = pickle.load(f)
    return data

In [3]:
# load the file
station_group = read_pkl("Aal/station_final.pkl")

In [4]:
pd.set_option("display.max_columns",station_group['Gade'].shape[1]+1)
station_group['Gade']

,station_id,NO2_Concentration,NOx_Concentration,time_year,time_month,time_week,time_day,time_hour,temperature,pressure,humidity,wind_direction,wind_speed,holiday,station_index,pressure_temperature,temperature_humidity,NO2*NOx,NO2_NOx,mean_NO2,mean_NOx,mean_temperature,mean_pressure,mean_humidity,mean_wind_speed,mean_wind_direction,max_NO2,max_NOx,max_temperature,max_pressure,max_humidity,max_wind_speed,var_NO2,var_NOx,var_temperature,var_pressure,var_humidity,var_wind_speed,var_wind_direction
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-06 10:00:00,Gade,11.84,18.23,2019,12,4,6,10,9.0,992.65,87.0,260.0,31.0,0,0,8933.85,0.103436,30.07,0.649123,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 10:30:00,Gade,17.29,26.09,2019,12,4,6,10,9.0,993.65,87.0,260.0,33.0,0,0,8942.85,0.103436,43.38,0.662452,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 11:00:00,Gade,21.40,31.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,33.0,0,0,7949.20,0.086012,53.20,0.672744,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 11:30:00,Gade,16.03,28.80,2019,12,4,6,11,8.0,993.65,93.0,250.0,30.0,0,0,7949.20,0.086012,44.83,0.556404,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2019-12-06 12:00:00,Gade,25.11,38.67,2019,12,4,6,12,8.0,993.65,93.0,250.0,24.0,0,0,7949.20,0.086012,63.78,0.649173,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-20 08:00:00,Gade,7.27,14.39,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0,0,0.00,0.000000,21.66,0.504861,12.710804,22.916607,0.0,0.0,0.0,0.0,0.0,38.89,106.38,0.0,0.0,0.0,0.0,68.182141,357.905090,0.0,0.0,0.0,0.0,0.0
2020-07-20 08:30:00,Gade,8.90,17.27,2020,7,0,20,8,0.0,0.00,0.0,0.0,0.0,0,0,0.00,0.000000,26.17,0.515046,12.716161,22.926161,0.0,0.0,0.0,0.0,0.0,38.89,106.38,0.0,0.0,0.0,0.0,68.131640,357.766441,0.0,0.0,0.0,0.0,0.0
2020-07-20 09:00:00,Gade,9.32,19.16,2020,7,0,20,9,0.0,0.00,0.0,0.0,0.0,0,0,0.00,0.000000,28.48,0.486176,12.717411,22.935149,0.0,0.0,0.0,0.0,0.0,38.89,106.38,0.0,0.0,0.0,0.0,68.122623,357.671515,0.0,0.0,0.0,0.0,0.0


In [5]:
def flatten_Y(features, ratio = 1, stations = list(station_group.keys()), start = 14 * 24):  
    
    if(start == 14 * 24):
        length = int(12000 * ratio)
    else:
        length = 10806 - 72
    
    Y_NOx_list = []
    Y_NO2_list = []
    X_extend_list = []
    
    for name in station_group:
        if name in stations:
            # print(name)
            NOx_list = station_group[name]['NOx_Concentration'].values
            NO2_list = station_group[name]['NO2_Concentration'].values
            
            temperature_list = station_group[name]['temperature'].values
            pressure_list = station_group[name]['pressure'].values
            humidity_list = station_group[name]['humidity'].values
            wind_speed_list = station_group[name]['wind_speed'].values
            wind_direction_list = station_group[name]['wind_direction'].values
            
            original_NOx_list = []
            original_NO2_list = []
            original_temperature_list = []
            original_pressure_list = []
            original_humidity_list = []
            original_wind_speed_list = []
            original_wind_direction_list = []
            
            X_this = station_group[name][features].values # features_matrix
            
            for i in range(start, length):
                Y_NOx_list.append(NOx_list[i+1 : i+49])
                Y_NO2_list.append(NO2_list[i+1 : i+49])
                
                differ_NOx_array = []
                differ_NO2_array = []
                differ_temperature_array = []
                differ_pressure_array = []
                differ_humidity_array = []
                differ_wind_speed_array = []
                differ_wind_direction_array = []
                
                original_NOx_array = NOx_list[i : i+48]
                original_NO2_array = NO2_list[i : i+48]
                original_temperature_array = temperature_list[i : i+72]
                original_pressure_array = pressure_list[i : i+72]
                original_humidity_array = humidity_list[i : i+72]
                original_wind_speed_array = wind_speed_list[i : i+72]
                original_wind_direction_array = wind_direction_list[i : i+72]
                
                for j in range(47):
                    differ_NOx_array.append(original_NOx_array[j+1] - original_NOx_array[j])
                    differ_NO2_array.append(original_NO2_array[j+1] - original_NO2_array[j])
                    
                for j in range(71):    
                    differ_temperature_array.append(original_temperature_array[j+1] - original_temperature_array[j])
                    differ_pressure_array.append(original_pressure_array[j+1] - original_pressure_array[j])
                    differ_humidity_array.append(original_humidity_array[j+1] - original_humidity_array[j])
                    differ_wind_speed_array.append(original_wind_speed_array[j+1] - original_wind_speed_array[j])
                    differ_wind_direction_array.append(original_wind_direction_array[j+1] - original_wind_direction_array[j])
                
                this_NOx = original_NOx_array[0]
                this_NO2 = original_NO2_array[0]
                this_temperature = original_temperature_array[0]
                this_pressure = original_pressure_array[0]
                this_humidity = original_humidity_array[0]
                this_wind_speed = original_wind_speed_array[0]
                this_wind_direction = original_wind_direction_array[0]
                processed_NOx_list = []
                processed_NO2_list = []
                processed_temperature_list = []
                processed_pressure_list = []
                processed_humidity_list = []
                processed_wind_speed_list = []
                processed_wind_direction_list = []
                
                for j in range(47):
                    random_differ = (random_sample((1)) + 0.5)  * differ_NOx_array[j]
                    if(this_NOx + random_differ >= 0):
                        this_NOx = np.around(this_NOx +  random_differ, decimals=1)
                    processed_NOx_list.append(this_NOx)
                    
                    # random_differ = (random_sample((1)) + 0.5) * differ_PM10_array[j]
                    # if(this_PM10 + random_differ >= 0):
                    #     this_PM10 = np.around(this_PM10 +  random_differ, decimals=1)
                    # processed_PM10_list.append(this_PM10)
                    
                    random_differ = (random_sample((1)) + 0.5) * differ_NO2_array[j]
                    if(this_NO2 + random_differ >= 0):
                        this_NO2 = np.around(this_NO2 +  random_differ, decimals=1)
                    processed_NO2_list.append(this_NO2)
                    
                for j in range(71):
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_temperature_array[j]
                    this_temperature = np.around(this_temperature +  random_differ, decimals=1)
                    processed_temperature_list.append(this_temperature)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_pressure_array[j]
                    this_pressure = np.around(this_pressure +  random_differ, decimals=1)
                    processed_pressure_list.append(this_pressure)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_humidity_array[j]
                    this_humidity = np.around(this_humidity +  random_differ, decimals=1)
                    processed_humidity_list.append(this_humidity)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_wind_speed_array[j]
                    this_wind_speed = np.around(this_wind_speed +  random_differ, decimals=1)
                    processed_wind_speed_list.append(this_wind_speed)
                    
                    random_differ = ((random_sample((1)) - 0.5) / 5 + 1) * differ_wind_direction_array[j]
                    this_wind_direction = np.around(this_wind_direction +  random_differ, decimals=1)
                    processed_wind_direction_list.append(this_wind_direction)
                
                processed_NOx = np.vstack(processed_NOx_list).flatten()
                processed_NO2 = np.vstack(processed_NO2_list).flatten()
                processed_temperature = np.vstack(processed_temperature_list).flatten()
                processed_pressure = np.vstack(processed_pressure_list).flatten()
                processed_humidity = np.vstack(processed_humidity_list).flatten()
                processed_wind_speed = np.vstack(processed_wind_speed_list).flatten()
                processed_wind_direction = np.vstack(processed_wind_direction_list).flatten()
                
                historical_NOx_array = NOx_list[i-11 : i+1]
                historical_NO2_array = NO2_list[i-11 : i+1]
                historical_temperature_array = temperature_list[i-11 : i+1]
                historical_pressure_array = pressure_list[i-11 : i+1]
                historical_humidity_array = humidity_list[i-11 : i+1]
                historical_wind_speed_array = wind_speed_list[i-11 : i+1]
                historical_wind_direction_array = wind_direction_list[i-11 : i+1]
                
                NOx_array = np.hstack((historical_NOx_array, processed_NOx))
                NO2_array = np.hstack((historical_NO2_array, processed_NO2))
                temperature_array = np.hstack((historical_temperature_array, processed_temperature))
                pressure_array = np.hstack((historical_pressure_array, processed_pressure))
                humidity_array = np.hstack((historical_humidity_array, processed_humidity))
                wind_speed_array = np.hstack((historical_wind_speed_array, processed_wind_speed))
                wind_direction_array = np.hstack((historical_wind_direction_array, processed_wind_direction))
                
                for j in range(48):
                    # if you do not want to get the feature of historical PM2.5, you can Comment the following code
                    tmp = NOx_array[j:j+12]
                    
                    # if you do not want to get the feature of historical NO2, you can Comment the following code
                    tmp = np.hstack((tmp, NO2_array[j:j+12]))
                    
                    tmp = np.hstack((tmp, temperature_array[j:j+24]))
                    tmp = np.hstack((tmp, pressure_array[j:j+24]))
                    tmp = np.hstack((tmp, humidity_array[j:j+24]))
                    tmp = np.hstack((tmp, wind_speed_array[j:j+24]))
                    tmp = np.hstack((tmp, wind_direction_array[j:j+24]))
                    tmp = np.append(tmp, j)
                    X_extend_list.append(np.hstack((X_this[i, :], tmp)))
            
    Y_NOx = np.vstack(Y_NOx_list)
    Y_NO2 = np.vstack(Y_NO2_list)
    Y_NOx = Y_NOx.flatten()
    Y_NO2 = Y_NO2.flatten()
    X = np.vstack(X_extend_list)
    
    return X, Y_NOx, Y_NO2    

In [6]:
def generate_train_data (features, k=1, name = 'all', start = 14 * 24):
    s = ["Gade","Tag"]
    X, Y_NOx, Y_NO2 = flatten_Y(features,ratio = k,stations = s, start = start)
    print('Generated data shape: ',X.shape, Y_NOx.shape,Y_NO2.shape)
    # save all of the X Y into .npy file
    np.save("Aal/X_"+name+".npy", X)
    np.save("Aal/Y_NO2_"+name+".npy", Y_NO2)
    np.save("Aal/Y_NOx_"+name+".npy", Y_NOx)
    print('Finish creating and saving the '+name+' version of X, Y_NO2, Y_PM10, and Y_PM25 data')

In [7]:
# If you want to removing some features in the generated X Y file (.npy), you can delete some of the following elements
# And [a,b] is differ from [b,a], which means you can control the feature order of the output data

features = ['NOx_Concentration','NO2_Concentration', # feature 0-1
            'temperature','pressure','humidity','wind_speed', 'wind_direction', # feature 2-6
            'holiday','time_month','time_week','time_day','time_hour','station_index', # feature 7-12
            
            'mean_NOx','mean_NO2','mean_temperature','mean_pressure','mean_humidity','mean_wind_speed','mean_wind_direction', # feature 13-19
            'max_NOx','max_NO2','max_temperature','max_pressure','max_humidity','max_wind_speed', # feature 20-26
            'var_NOx','var_NO2','var_temperature','var_pressure','var_humidity','var_wind_speed','var_wind_direction', # feature 27-33 hmmm two wind speed wonder if it should be directions
            'pressure_temperature','temperature_humidity','NO2*NOx','NO2_NOx' # feature 34-37 		
            
            # feature 37-48 historical NOx features
            # feature 49-60 historical NO2 features
            # feature 61-84 historical temperature and forecasting features
            # feature 85-108 historical pressure and forecasting features
            # feature 109-132 historical humidity and forecasting features
            # feature 133-156 historical wind_speed and forecasting features
            # feature 157-180 historical wind_direction and forecasting features
            
            # feature 181: prediction id
            ]

In [8]:
# Generate a testing dataset of X Y data (.npy)
# The generated file name is X_test.npy, etc.
generate_train_data(features, name = 'test', start = 350)

Generated data shape:  (996864, 182) (996864,) (996864,)
Finish creating and saving the test version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [9]:
# Generate a small-sized version of X Y data (.npy)
# The generated file name is X_s.npy, etc.
generate_train_data(features, k = 0.08, name = 's')
# Reminder: k must be greater than 0.0374

Generated data shape:  (59904, 182) (59904,) (59904,)
Finish creating and saving the s version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [10]:
# You can also generate a small-sized version of X Y data (.npy) with removing some features.

In [10]:
# Generate a medium-sized version of X Y data (.npy)
# The generated file name is X_m.npy, etc.
generate_train_data(features, k = 0.2, name = 'm')

Generated data shape:  (198144, 182) (198144,) (198144,)
Finish creating and saving the m version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [12]:
# You can also generate a medium-sized version of X Y data (.npy) with removing some features.

In [12]:
# Generate a whole-sized version of X Y data (.npy)
# The generated file name is X_all.npy, etc.
generate_train_data(features, k = 0.9, name = 'all')

Generated data shape:  (1004544, 182) (1004544,) (1004544,)
Finish creating and saving the all version of X, Y_NO2, Y_PM10, and Y_PM25 data


In [14]:
# If you want to generate X Y files with other small size, you can change the ratio element
# For example, if you change the input ratio 'k' to 0.3 in the generate_train_data function, then you can get the X Y files with 30% of the whole size